# Working with MongoDB
- You need to install [MongoDB](https://www.mongodb.com/) to use this notebook.

In [2]:
const MongoClient = require("mongodb").MongoClient

In [3]:
const dbName = "nodejs-learners-package-db";
const dbPort = '27017';
const uri = `mongodb://localhost/:${27017}/${dbName}`; // Replace with your actual connection string
const client = new MongoClient(uri);